# 

In [177]:
# 目的
# S21を定数スイープしたデータ
# ピーク値とイメージ、ハーフを抽出すること

In [55]:
import pandas as pd
import numpy as np

# ファイル名
fname = "himg_fM.csv"
fname_out = "himg_fM_out.csv"

In [179]:
# 【フロー】
# ① CSVをカラム名付けて読み込む=>完了
# ② T2の数値ごとに分ける [fL, fM, fH]=>完了
# ③ 各定数値のS21ピーク値を読む [gain]=>完了
# ④ イメージ、ハーフイメージの周波数を読む=>完了
# ⑤ ピーク値と比を取って特性値とする。この際、300MHz以下、650MHz以上は"NA"と表記する=>完了
# ⑥ 上記をループで全部回す=>完了
# ⑦ 最終結果をCSVで出力する

In [180]:
# 最終データのイメージ
# C205, T2, fpeak, gain, fimg, img, fhimg, himg

In [181]:
# S21がピーク値を取るときのFreqの組み合わせ
# 300MHz - 650MHz
# 100MHz - 800MHz

# 【スイープ範囲】
# T2 = [1,2.5,4]の3ステップ
# C = 0.25-50の51ステップ

# 【周波数範囲】
# 300000000.0 - 650000000.0

In [56]:
# ヘッダを加える
# df = pd.read_csv(fname, header = None, names = ["ID", "Val", "T12", "Freq", "S21"])
df = pd.read_csv(fname, header = None, names = ["ID","Val1","Val2","Val3","T12","Freq","S21"])

# 先頭行と末尾行を削除する
df = df.drop(index=0)
df = df.drop(index=df.shape[0])

# IDは不要のため抹消する
df = df.drop(columns="ID")

In [12]:
df

,Val1,Val2,Val3,T12,Freq,S21
1,3.7,5.7,26.7,1.0,100000000.0,-94.796616
2,3.7,5.7,26.7,1.0,101000000.0,-94.349016
3,3.7,5.7,26.7,1.0,102000000.0,-93.907216
4,3.7,5.7,26.7,1.0,103000000.0,-93.471069
5,3.7,5.7,26.7,1.0,104000000.0,-93.040438
...,...,...,...,...,...,...
757076,4.3,6.3,27.3,1.4,796000000.0,-32.669449
757077,4.3,6.3,27.3,1.4,797000000.0,-32.705304
757078,4.3,6.3,27.3,1.4,798000000.0,-32.741077
757079,4.3,6.3,27.3,1.4,799000000.0,-32.776770


In [183]:
# T2ごとに区分けする
# fL = df.loc[df["T2"]==1.0]
# fM = df.loc[df["T2"]==2.5]
# fH = df.loc[df["T2"]==4.0]

In [57]:
df_all = pd.DataFrame(columns=["Val1","Val2","Val3","T12","fpeak","gain","fimg","img","fhimg","himg"])

# T12: 1.0-1.4,0.1 : 2.2-2.6,0.1: 3.2-3.6,0.1
# Tf = [1.0, 1.1, 1.2, 1.3, 1.4]
Tf = [2.2, 2.3, 2.4, 2.5, 2.6]
# Tf = [3.2, 3.3, 3.4, 3.5, 3.6]
# [fpeak,gain]
# L102: 26.7-27.3,0.12
# C107: 5.7-6.3,0.12
# C108: 3.7-4.3,0.12
# Val3 = [26.7,26.82,26.94,27.06,27.18,27.3] # L102
# Val2 = [5.7,5.82,5.94,6.06,6.18,6.3]
# Val1 = [3.7,3.82,3.94,4.06,4.18,4.3]

step1 = 6
step2 = 6
step3 = 6

# val3max = 27.3 # L102
# val3min = 26.7
# val3step = 0.12
# val2max = 6.3 # C107
# val2min = 5.7
# val2step = 0.12
# val1max = 4.3 # C108
# val1min = 3.7
# val1step = 0.12
#------------------------------
# [img]
# L102: 26.7-27.3,0.12
# C105: 1.7-2.3,0.12
# C107: 5.7-6.3,0.12
# Val3 = [26.7,26.82,26.94,27.06,27.18,27.3] # L102
# Val2 = [1.7,1.82,1.94,2.06,2.18,2.3]
# Val1 = [5.7,5.82,5.94,6.06,6.18,6.3]

# val1max = 27.3 # L102
# val1min = 26.7
# val1step = 6
# val2max = 2.3 # C105
# val2min = 1.7
# val2step = 6
# val3max = 6.3 # C107
# val3min = 5.7
# val3step = 6
#------------------------------
# [himg]
# L102: 26.7-27.3,0.12
# C108: 3.7-4.3,0.12
# C107: 5.7-6.3,0.12
Val3 = [26.7,26.82,26.94,27.06,27.18,27.3] # L102
Val2 = [3.7,3.82,3.94,4.06,4.18,4.3]
Val1 = [5.7,5.82,5.94,6.06,6.18,6.3]

# val1max = 27.3 # L102
# val1min = 26.7
# val1step = 6
# val2max = 4.3 # C108
# val2min = 3.7
# val2step = 6
# val3max = 6.3 # C107
# val3min = 5.7
# val3step = 6

# cmax = 50
# cmin = 0.25
# step = 51

# T2調整値を選択
for i, val1 in enumerate(Val1):

    # 1つ目のパラメータ
    for j, val2 in enumerate(Val2):
        # 2つ目のパラメータ
        for k, val3 in enumerate(Val3):
            # 3つ目のパラメータ
            for l, T in enumerate(Tf):

                # ループで使用するデータフレームを抽出
                df_tmp = df.loc[df["Val1"]==val1]
                df_tmp = df_tmp.loc[df_tmp["Val2"]==val2]
                df_tmp = df_tmp.loc[df_tmp["Val3"]==val3]
                df_tmp = df_tmp.loc[df_tmp["T12"]==T]

                # ピーク値を取る
                tmp = df_tmp[df_tmp["S21"] == df_tmp["S21"].max()]
                fpeak = tmp["Freq"].tolist()[0]
                gain = tmp["S21"].tolist()[0]

                fimg = fpeak - 100e6
                fhimg = fpeak - 25e6

                # イメージの計算
                if(fimg>100e6):
                    img = gain - df_tmp[df_tmp["Freq"]==fimg]["S21"].tolist()[0]
                else:
                    img = "NG"

                # ハーフイメージの計算
                if(fhimg>100e6):
                    himg = gain - df_tmp[df_tmp["Freq"]==fhimg]["S21"].tolist()[0]
                else:
                    img = "NG"
                
                # row = l + val1step*k + val2step*j  + val3step*i 
                # row = step1*j + step2*k + step3*l + i
                row = step3*(k+step2*(j+step3*i)) + l

                # print(row)
                # 元データに挿入
                df_all.loc[row,"Val1"] = val1
                df_all.loc[row,"Val2"] = val2
                df_all.loc[row,"Val3"] = val3
                df_all.loc[row,"T12"] = T
                df_all.loc[row,"fpeak"] = fpeak
                df_all.loc[row,"gain"] = gain
                df_all.loc[row,"fimg"] = fimg
                df_all.loc[row,"img"] = img
                df_all.loc[row,"fhimg"] = fhimg
                df_all.loc[row,"himg"] = himg
                # break
            # break
        # break
    # break


In [58]:
df_all.to_csv(fname_out)